# Model Controller Tutorial: Roberta model with Deep Hierarchical Classification

> This notebook contains some example of how to use the Roberta-based models in this NLP library

- skip_showdoc: true
- skip_exec: true

In this tutorial, we walk through another special case of classification with multiple heads, which is based on this paper: https://arxiv.org/ftp/arxiv/papers/2005/2005.06692.pdf

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
from transformers import RobertaTokenizer
from datasets import Dataset

# Define the custom augmentation function

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Department Name'],
                         sup_types=['classification','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

Define our tokenizer for Roberta

In [ ]:
_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Filter (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Flattening the indices (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18101
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

# Model Experiment: Roberta Multi-Head Classification (with Hidden Layer Concatenation)

In [ ]:
from that_nlp_library.models.roberta.deep_hierarchical_classifiers import *
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import f1_score, accuracy_score
from transformers.models.roberta.modeling_roberta import RobertaModel
import torch

comet_ml is installed but `COMET_API_KEY` is not set.


## Build DHC Conditional Mask

In [ ]:
tdc.label_names

['Division Name', 'Department Name']

In [ ]:
tdc.label_lists

[['General', 'General Petite', 'Initmates'],
 ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
df_trn = tdc.main_ddict['train'].to_pandas()

In [ ]:
df_labels = pd.DataFrame(df_trn['label'].tolist())
df_labels.columns=tdc.label_names

In [ ]:
df_labels.head()

,Division Name,Department Name
0,0,4
1,1,1
2,1,1
3,1,3
4,0,1


In [ ]:
dhc_mask = build_DHC_conditional_mask(df_labels,*tdc.label_names)

In [ ]:
dhc_mask.shape

torch.Size([3, 6])

In [ ]:
dhc_mask

tensor([[1., 1., 0., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 0.]])

In [ ]:
tdc.label_lists

[['General', 'General Petite', 'Initmates'],
 ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

Explain the first row of the mask (for the first label of `Division Name`: `General`)

In [ ]:
dhc_mask[0]

tensor([1., 1., 0., 1., 1., 1.])

Slicing the first portion for `Department Name`, show string for True mask. The results are the sub-category of `Division Name`

In [ ]:
for i in torch.where(dhc_mask[0]==True)[0]:
    print(tdc.label_lists[1][i])

Bottoms
Dresses
Jackets
Tops
Trend


In [ ]:
# let's double check with the original data
np.sort(df_trn[df_trn['Division Name']=='General']['Department Name'].unique())

array(['Bottoms', 'Dresses', 'Jackets', 'Tops', 'Trend'], dtype=object)

## Define and train a custom Roberta model

### Full model

In [ ]:
roberta_body = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
_model_kwargs={
    'dhc_mask':dhc_mask,
    'classifier_dropout':0.1,
    'last_hidden_size':768,  
    'linear_l1_size':389,
    'linear_l2_size':417,
    'lloss_weight':1.0,
    'dloss_weight':0.8,
    'layer2concat':4,
}

model = model_init_classification(model_class = RobertaHSCDHCSequenceClassification,
                                  cpoint_path = 'roberta-base', 
                                  output_hidden_states=True,
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

Loading body weights. This assumes the body is the very first block of your custom architecture


And we can start training our model

In [ ]:
seed_everything(42)

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_separate_heads,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score Division name,Accuracy Score Division name,F1 Score Department name,Accuracy Score Department name
1,No log,3.512304,0.429403,0.615996,0.672249,0.875608
2,3.647000,3.471539,0.520462,0.579099,0.667668,0.878259
3,3.647000,3.441603,0.449887,0.618648,0.682390,0.883562


### Simpler model

In [ ]:
seed_everything(42)

In [ ]:
roberta_body = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
_model_kwargs={
    'dhc_mask':dhc_mask,
    'lloss_weight':1.0,
    'dloss_weight':0.8,
    'layer2concat':4,
}

model = model_init_classification(model_class = RobertaSimpleHSCDHCSequenceClassification,
                                  cpoint_path = 'roberta-base', 
                                  output_hidden_states=True,
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

Loading body weights. This assumes the body is the very first block of your custom architecture


And we can start training our model

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_separate_heads,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score Division name,Accuracy Score Division name,F1 Score Department name,Accuracy Score Department name
1,No log,3.556299,0.434068,0.611798,0.655985,0.866991
2,3.672200,3.498545,0.443362,0.617101,0.680612,0.882457
3,3.672200,3.468069,0.459212,0.620415,0.682411,0.883562


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model')

## Make predictions

### Load trained model

In [ ]:
_model_kwargs

{'dhc_mask': tensor([[1., 1., 0., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 0., 0., 0.]]),
 'lloss_weight': 1.0,
 'dloss_weight': 0.8,
 'layer2concat': 4}

In [ ]:
trained_model = model_init_classification(model_class = RobertaSimpleHSCDHCSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

Some weights of the model checkpoint at sample_weights/my_model were not used when initializing RobertaSimpleHSCDHCSequenceClassification: ['body_model.pooler.dense.bias', 'body_model.pooler.dense.weight']
- This IS expected if you are initializing RobertaSimpleHSCDHCSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaSimpleHSCDHCSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Predict Train/Validation set

In [ ]:
df_val = controller.predict_ddict(ds_type='validation',are_heads_separated=True)

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Division Name,Department Name,label,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,,. such a fun jacket ! great to wear in the spr...,General Petite,Intimate,"[1, 2]","[0, 4, 215, 10, 1531, 8443, 27785, 372, 7, 356...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.559910,Jackets,0.789852
1,simple and elegant,simple and elegant . i thought this shirt was ...,General Petite,Tops,"[1, 4]","[0, 41918, 8, 14878, 479, 939, 802, 42, 6399, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.673332,Tops,0.985734
2,retro and pretty,retro and pretty . this top has a bit of a ret...,General,Tops,"[0, 4]","[0, 4903, 1001, 8, 1256, 479, 42, 299, 34, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.649570,Tops,0.985392
3,summer/fall wear,summer / fall wear . i first spotted this on a...,General Petite,Dresses,"[1, 1]","[0, 18581, 2089, 1589, 1136, 3568, 479, 939, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.563204,Dresses,0.972883
4,perfect except slip,perfect except slip . this is my new favorite ...,General Petite,Dresses,"[1, 1]","[0, 20473, 4682, 9215, 479, 42, 16, 127, 92, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.579009,Dresses,0.981901


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val['Division Name'],df_val['pred_Division Name'],average='macro')

0.45921193659675547

In [ ]:
f1_score(df_val['Department Name'],df_val['pred_Department Name'],average='macro')

0.6824108822326193

### Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)

# save the label, as we will calculate some metrics later. We also filter out labels with NaN Review Text,
# as there will be a filtering processing on the test set
true_labels = df_test.loc[~df_test['Review Text'].isna(),'Department Name'].values 

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop(['Division Name','Department Name'],axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True,
                                                   are_heads_separated=True,
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[0, 20473, 13, 173, 8, 310, 479, 42, 6399, 136...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.663096,Tops,0.988319
1,,. i don't know why i had the opposite problem ...,"[0, 4, 939, 218, 75, 216, 596, 939, 56, 5, 548...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.729423,Bottoms,0.990833
2,great pants,great pants . thes e cords are great--lightwei...,"[0, 12338, 9304, 479, 5, 29, 364, 37687, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.714403,Bottoms,0.989392
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[0, 33258, 3137, 24382, 13, 10, 6148, 159, 479...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.587568,Tops,0.947718
4,short and small,short and small . the shirt is mostly a thick ...,"[0, 20263, 8, 650, 479, 5, 6399, 16, 2260, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.650256,Tops,0.978389


Let's quickly check the f1 score to make sure everything works correctly

In [ ]:
f1_score(true_labels,df_test_predicted['pred_Department Name'],average='macro')

0.7029243366696271

Predict top k results

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True,
                                                   topk=3,
                                                   are_heads_separated=True
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[0, 20473, 13, 173, 8, 310, 479, 42, 6399, 136...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.66309553, 0.32625943, 0.010645005]","[Tops, Intimate, Jackets]","[0.9883187, 0.009802946, 0.0010778408]"
1,,. i don't know why i had the opposite problem ...,"[0, 4, 939, 218, 75, 216, 596, 939, 56, 5, 548...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.72942257, 0.26586807, 0.004709378]","[Bottoms, Intimate, Dresses]","[0.990833, 0.0060857264, 0.0027363764]"
2,great pants,great pants . thes e cords are great--lightwei...,"[0, 12338, 9304, 479, 5, 29, 364, 37687, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.7144031, 0.2791949, 0.0064020962]","[Bottoms, Intimate, Dresses]","[0.9893918, 0.008282459, 0.0019725526]"
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[0, 33258, 3137, 24382, 13, 10, 6148, 159, 479...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.5875683, 0.36433047, 0.0481012]","[Tops, Intimate, Jackets]","[0.94771826, 0.044398014, 0.00635766]"
4,short and small,short and small . the shirt is mostly a thick ...,"[0, 20263, 8, 650, 479, 5, 6399, 16, 2260, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.65025616, 0.3250219, 0.024721952]","[Tops, Intimate, Jackets]","[0.97838897, 0.019185867, 0.001521599]"


In [ ]:
# Since we have some metadatas (Title), we need to define a dictionary containing those values
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt'}

In [ ]:
controller.data_store.num_proc=1

In [ ]:
df_result = controller.predict_raw_text(raw_content,are_heads_separated=True,topk=3)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
df_result

,Review Text,Title,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,great shirt . this shirt is so comfortable i l...,great shirt,"[0, 12338, 6399, 479, 42, 6399, 16, 98, 3473, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[General, General Petite, Initmates]","[0.65943, 0.32565758, 0.014912433]","[Tops, Intimate, Jackets]","[0.9834737, 0.014790922, 0.000995291]"
